In [ ]:
# Artificial Neural Network

### Importing the libraries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.style.use('dark_background')
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.metrics import plot_confusion_matrix
from scipy.stats import norm, boxcox
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from collections import Counter
from scipy import stats
import tensorflow as tf
import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Importing the dataset

In [ ]:
dataset = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')

# Exploratory Data Analysis

In [ ]:
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.describe().T.style.bar(
    subset=['mean'],
    color='#606ff2').background_gradient(
    subset=['std'], cmap='PuBu').background_gradient(subset=['50%'], cmap='PuBu')

In [ ]:
dataset.isnull().values.any()

## Plotting Count for Qualities

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(x="Exited", data=dataset, palette='husl');

## Some BoxPlots

In [ ]:
def boxPlotter(columnName):
        sns.catplot(x="Exited", y=columnName, data=dataset, kind="box");

In [ ]:
cols = ['CreditScore', 'Age', 'Tenure', 'Balance', 'EstimatedSalary']
for column in cols:
    boxPlotter(column)

From above Plots we can conclude that:-
1. There is no significant difference in credit score distribution between customers which are churned or not.
1. The older customers are churning more than younger ones.
1. Bank is loosing customers with significant bank balance.
1. Estimated Salary does not have a significant on the likelihood to churn.
Interestingly, majority of customers that churned are those with credit cards but this can be a coincidence as majority of customers have credit cards.
Unsurprisingly the inactive members have a greater churn and the overall proportion of inactive members is also very high.

## Correlation Plot

In [ ]:
plt.figure(figsize=(20, 17))
matrix = np.triu(dataset.corr())
sns.heatmap(dataset.corr(), annot=True,linewidth=.8, mask=matrix, cmap="rocket");

## From above correlation plot we can say that there is no multicolinearity present in data

## Some PieCharts

In [ ]:
plot_data =  dataset.drop(['RowNumber', 'CustomerId', 'Surname','Gender','NumOfProducts', 'HasCrCard','Geography',
       'IsActiveMember', ], axis=1)

In [ ]:
def pieChartPlotter(dataset, columnName):
    values = dataset[columnName].value_counts()
    labels = dataset[columnName].unique()
    pie, ax = plt.subplots(figsize=[10, 6])

    patches, texts, autotexts = ax.pie(values, labels=labels, autopct='%1.2f%%', shadow=True, pctdistance=.5,# explode=[0.06]*dataset['Exited'].unique()
                                       )

    plt.legend(patches, labels, loc="best")
    plt.title(columnName, color='white', fontsize=14)
    plt.setp(texts, color='white', fontsize=20)
    plt.setp(autotexts, size=10, color='black')
    autotexts[1].set_color('black')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

In [ ]:
pieChartPlotter(dataset, 'Exited') 
pieChartPlotter(dataset, 'Gender')
pieChartPlotter(dataset, 'Geography')

### From Above plot
1. We can see that 79.63% of customers have churned out 
1. There are 54.57% Males and 45.43% females
1. About 50.14% customers are from France, 25.09% from Spain and 24.77% from germany

## Group Plots

In [ ]:
def groupBarPlotter(dataset):
    """
    Group Plots columns with Exited column
    """
    fig = plt.figure(figsize=(20, 30))
    for i in range(len(dataset.columns)):
        if not dataset.columns[i] == 'Exited':
            groups = dataset.groupby(dataset.columns[i])['Exited'].mean()
            fig.add_subplot(np.ceil(len(dataset.columns)/2), 2, i+1)
            plt.xlabel('price')
            groups.plot.barh()
            fig.tight_layout(pad=3.0)

In [ ]:
groupBarPlotter(dataset.loc[:, ['Tenure', 'Age','Exited']])

## Distribution Plot

In [ ]:
def distributionPlot(dataset):
    fig = plt.figure(figsize=(20, 20))
    for i in range(len(dataset.columns)):
        fig.add_subplot(np.ceil(len(dataset.columns)/5), 3, i+1)
        sns.distplot(
            dataset.iloc[:, i], color="lightcoral", rug=True)
        fig.tight_layout(pad=3.0)

In [ ]:
distributionPlot(plot_data)

## Pairplots

In [ ]:
sns.pairplot(plot_data, hue="Exited", palette="husl");

### From above plots we can say there's skewness in the data

## Skewness Correction

In [ ]:
def skewnessCorrector(dataset,columnName):
    import seaborn as sns
    from scipy import stats
    from scipy.stats import norm, boxcox
    """
    This function returns two plots distplot and probability plot for non-normalized data and after normalizing the provided data. 
    Just provide it with two parameters dataset and the name of column.
    It corrects the skewness of data applying Boxcox transformation on the provided data
    """
    print('''Before Correcting''')
    (mu, sigma) = norm.fit(dataset[columnName])
    print("Mu before correcting {} : {}, Sigma before correcting {} : {}".format(
        columnName.capitalize(), mu, columnName.capitalize(), sigma))
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    sns.distplot(dataset[columnName], fit=norm, color="lightcoral");
    plt.title(columnName.capitalize() +
              " Distplot before Skewness Correction", color="black")
    plt.subplot(1, 2, 2)
    stats.probplot(dataset[columnName], plot=plt)
    plt.show()
    # Applying BoxCox Transformation
    dataset[columnName], lam_fixed_acidity = boxcox(
        dataset[columnName])
    
    print('''After Correcting''')
    (mu, sigma) = norm.fit(dataset[columnName])
    print("Mu after correcting {} : {}, Sigma after correcting {} : {}".format(
        columnName.capitalize(), mu, columnName.capitalize(), sigma))
    plt.figure(figsize=(20, 10))
    plt.subplot(1, 2, 1)
    sns.distplot(dataset[columnName], fit=norm, color="orange");
    plt.title(columnName.capitalize() +
              " Distplot After Skewness Correction", color="black")
    plt.subplot(1, 2, 2)
    stats.probplot(dataset[columnName], plot=plt)
    plt.show()


In [ ]:
skewed_columns = ['CreditScore', 'Age', 'EstimatedSalary']
for column in skewed_columns:
    skewnessCorrector(dataset,column)

# Data Preprocessing

In [ ]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

In [ ]:
print(y)

## Encoding categorical data

### Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

### One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .2, random_state= 0)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
print(y_test)

In [ ]:
print(y_train)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

# Building the ANN

### Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()


### Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding hidden layers

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Training the ANN

### Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'],)

### Training the ANN on the Training set

In [ ]:
ann_trained = ann.fit(X_train, y_train,validation_split = 0.20, validation_data = (X_test, y_test), epochs = 100)

## Making the predictions and evaluating the model

### Predicting the result of a single observation

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > .5)

Therefore, our ANN model predicts that this customer leaves the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

## Plotting Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
ax= plt.subplot()
sns.heatmap(cm, annot=True, fmt='g', ax=ax);  
ax.set_xlabel('Predicted labels');
ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['0', '1']); 
ax.yaxis.set_ticklabels(['0', '1']);


## Classification Report

In [ ]:
print(classification_report(y_test, y_pred,
          target_names=['0', '1'], zero_division=1))

In [ ]:
print('Accuracy of Model is')
accuracy_score(y_test, y_pred)

### Our Ann's Accuracy is 85.95%

# Visualising Our ANN's Loss and Accuracy

In [ ]:
def annPlotter(model, plotType):
    plt.figure(figsize = (12, 6))
    if plotType == 'loss':
        param_1 = 'loss'
        param_2 = 'val_loss'
    elif plotType == 'accuracy':
        param_1 = 'accuracy'
        param_2 = 'val_accuracy'
    train = model.history[param_1]
    val = model.history[param_2] 
    epoch = range(1, 101)
    sns.lineplot(epoch, train, label = 'Training '+plotType.capitalize())
    sns.lineplot(epoch, val, label = 'Validation '+plotType.capitalize())
    plt.title('Training and Validation '+plotType.capitalize())
    plt.xlabel('Epochs')
    plt.ylabel(plotType.capitalize())
    plt.legend()
    plt.show()

In [ ]:
annPlotter(ann_trained, 'loss')
annPlotter(ann_trained, 'accuracy')

## Thanks for Reading
---
## Please share your feedback by commenting below and if you like my work please consider upvoting.